In [110]:
import numpy as np 
from janome.tokenizer import Tokenizer
from gensim.models.keyedvectors import KeyedVectors
import torch
from torch import nn, optim
import pickle
from sys import argv
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import  torch.nn.functional as F

In [42]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
!cat './tanaka_corpus/train.en.vocab.4k'

<unk>
<s>
</s>
.
the
i
to
you
is
he
a
?
in
it
of
she
for
my
have
this
't
was
me
,
that
at
his
we
do
will
on
her
not
's
with
your
are
what
as
be
him
can
there
don
go
has
up
like
all
time
out
by
very
they
come
how
from
please
no
and
had
good
'm
am
here
did
get
about
'll
when
been
an
see
take
so
work
day
let
would
know
now
went
were
book
going
father
well
if
made
one
than
should
school
may
last
much
!
home
got
want
make
money
english
man
long
more
car
any
room
tomorrow
us
too
but
must
help
house
could
came
our
way
off
new
mother
yesterday
today
after
better
some
back
think
nothing
soon
before
where
always
many
who
every
old
just
put
look
little
people
night
two
down
late
give
hard
tell
say
next
over
morning
years
took
've
said
speak
never
right
cannot
station
didn
once
train
read
party
keep
into
does
plan
cold
walk
call
rain
three
teacher
boy
such
lot
play
bus
ten
children
week
left
away
live
leave
mind
happy
'd
again
tom
kind
accident
letter
anything
why
true
watch
only
job
something
new

In [3]:
!cat './tanaka_corpus/train.ja.vocab.4k'

<unk>
<s>
</s>
。
は
い
に
た
を
の
て
で
な
が
彼
し
る
私
す
っ
ま
か
だ
彼女
と
う
その
も
く
ん
、
あ
ら
こと
よ
この
き
り
する
お
せ
あなた
かっ
行
君
何
たち
から
それ
言
人
さ
ば
れ
よう
でき
へ
しょ
本
話
つ
くださ
見
くれ
家
ろ
仕事
どう
これ
だっ
時
いつ
や
もう
思
とても
車
知
まで
なさ
日
会
ここ
時間
来
もの
好き
わ
すぐ
ね
今
誰
そう
中
出
自分
下さ
金
英語
父
み
一
今日
達
僕
部屋
？
前
ため
けれ
あの
られ
学校
たら
子供
より
持
明日
よく
事
問題
雨
聞
電話
べ
さん
ところ
そんな
勉強
そこ
どこ
昨日
日本
年
方
いけ
少し
１
考え
手紙
だけ
読
書
必要
つもり
買
母
気
しま
我々
わか
待
たくさん
手
着
本当
住
先生
成功
良
駅
計画
病気
２
来る
バス
見え
目
早
む
度
悪
遅れ
帰
語
トム
食べ
パーティー
高
新し
ほど
働
降
出発
入
１０
楽し
すぎ
くらい
親切
事故
もら
ご
犬
まだ
少年
どの
じ
歩
ほとんど
３
長
医者
試験
死
なら
テレビ
起こ
出かけ
間
同じ
かか
教え
後
友達
使
ずっと
ず
忙し
みんな
とき
結婚
息子
テニス
もっと
忘れ
寝
時計
旅行
一番
通り
名前
男
起き
上
走
若
美し
列車
分
うま
宿題
ちょっと
飲
道
学生
健康
５
歌
助け
あまり
始め
出来
すべて
疲れ
川
アメリカ
なんて
約束
友人
音楽
町
一緒
うち
終え
一人
！
くる
立
東京
注意
水
上手
質問
取
ほう
分か
乗
必ず
どんな
あげ
答え
映画
かけ
また
終わ
大変
辞書
天気
多く
両親
どちら
家族
失敗
入れ
ジョン
コーヒー
意見
難し
以上
次
散歩
知らせ
だれ
兄
こ
手伝
決して
毎日
フランス
ドア
いただけ
急
出席
欲し
懸命
作
戻
つけ
しれ
者
なん
生徒
全く
不
寒
一生
メアリー
幸せ
はず
回
二
言葉
背
ひど
月
店
ばかり
すれ
大き
公園
やめ
朝
心配
夜
ぶ
午後
顔
試合
風邪
雪
愛
外国
呼
機
速
無
泣
わたし
間に合
通
なぜ
飛行
野球
週間
見つけ
外出
６
絵


In [10]:
id, eid2w, ew2id = 1, {}, {}
with open('./tanaka_corpus/train.en.vocab.4k', 'r', encoding='utf-8') as f:
    for w in f:
        w = w.strip() 
        eid2w[id] = w 
        ew2id[w] = id
        id +=1
ev = id

In [11]:
id, jid2w, jw2id = 1, {}, {}
with open('./tanaka_corpus/train.ja.vocab.4k', 'r', encoding='utf-8') as f:
    for w in f:
        w = w.strip() #文頭、文末の空欄・改行等の空白文字を削除
        jid2w[id] = w 
        jw2id[w] = id
        id +=1
jv = id

In [23]:
# 英語　⇒　文字から文字idへ変換

edata = []
with open('./tanaka_corpus/train.en', 'r', encoding='utf-8') as f:
    for sen in f:
        w1 = [ew2id['<s>']] # 各センテンスの始まりを指定
        for w in sen.strip().split():
            if w in ew2id: # 文字が vocabにあれば
                w1.append(ew2id[w])
            else:
                w1.append(ew2id['<unk>'])
        w1.append(ew2id['</s>'])　# 各センテンスの終わりを指定
        edata.append(w1)

In [27]:
# 日本語　⇒　文字から文字idへ変換

jdata = []
with open('./tanaka_corpus/train.ja', 'r', encoding='utf-8') as f:
    for sen in f:
        w1 = [jw2id['<s>']] # 各センテンスの始まりを指定
        for w in sen.strip().split():
            if w in jw2id:
                w1.append(jw2id[w])
            else:
                w1.append(jw2id['<unk>'])
        w1.append(jw2id['</s>']) # 各センテンスの終わりを指定
        jdata.append(w1)

In [48]:
# model

class MyMNT(nn.Module):
    def __init__(self, jv, ev, k):
        super().__init__()
        self.jemb = nn.Embedding(jv, k)
        self.eemb = nn.Embedding(ev, k)
        self.lstm1 = nn.LSTM(k, k, num_layers=2) # for encorder
        self.lstm2 = nn.LSTM(k, k, num_layers=2) # for decoder
        self.liner = nn.Linear(k, ev)

    def forward(self, jline, eline):
        x = self.jemb(jline)
        ox, (hnx, cnx) = self.lstm1(x)
        y = self.eemb(eline)
        oy, (hny, cny) = self.lstm2(y, (hnx, cnx))
        out = self.liner(oy)
        return out


In [49]:
demb = 200
net = MyMNT(jv, ev, demb).to(device)
optimizer = optim.SGD(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()

In [76]:
net.train()
for epoch in range(20):
    loss1k = 0.0
    for i in range(len(jdata)):
        jinput = torch.LongTensor(jdata[i][1:]).to(device) # <s>を除く
        einput = torch.LongTensor(edata[i][:-1]).to(device)
        out = net(jinput, einput)
        gans = torch.LongTensor([edata[i][1:]]).to(device) # decoderの出力は<s>の次なのではないので除く
        loss = criterion(out, gans[0])
        loss1k += loss.item()
        if (i % 100 == 0):
            print(epoch, i, loss1k)
            loss1k = 0

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    outfile = 'nmt.model'
    torch.save(net.state_dict(), outfile)

0 0 4.337253093719482
0 100 400.5509135723114
0 200 403.08455753326416
0 300 396.25804591178894
0 400 398.7655140161514
0 500 396.94766449928284
0 600 401.41981053352356
0 700 401.2605881690979
0 800 393.4293484687805
0 900 403.27519392967224
0 1000 398.82075119018555
0 1100 406.10607051849365
0 1200 404.94845843315125
0 1300 390.8788924217224
0 1400 393.5023868083954
0 1500 392.48692750930786
0 1600 408.44492626190186
0 1700 399.0980763435364
0 1800 394.68378806114197
0 1900 394.99736428260803
0 2000 395.796040058136
0 2100 398.14290285110474
0 2200 396.10163021087646
0 2300 396.7422158718109
0 2400 391.9548680782318
0 2500 399.34096693992615
0 2600 397.36270117759705
0 2700 395.77351546287537
0 2800 407.32268595695496
0 2900 388.32218730449677
0 3000 396.1204717159271
0 3100 405.2988085746765
0 3200 387.86980295181274
0 3300 408.04245018959045
0 3400 389.41882705688477
0 3500 405.11721527576447
0 3600 403.408408164978
0 3700 400.3824055194855
0 3800 401.9093692302704
0 3900 395.88439

# BLUEによる評価

In [90]:
from nltk.translate.bleu_score import corpus_bleu


In [88]:
!python3 './hidden_files/pytorch-nlp01r1/Chapter4/NMT/nmt-test.py' ./nmt.model

In [95]:
# blueを使用した評価

# 正解ラベル
labels = []
with open('./test.en', 'r', encoding='utf-8') as f:
    for sen in f:
        w = sen.strip().split()
        labels.append([w])

# モデルが予想した文字列
myans = []
with open('./output.txt', 'r', encoding='utf-8') as f:
    for sen in f:
        w = sen.strip().split()
        myans.append(w)

score = corpus_bleu(labels, myans)

print(score*100)



14.46114427578182


In [96]:
# attention

class MyAttNMT(nn.Module):
    def __init__(self, jv, ev, k):
        super(MyAttNMT, self).__init__()
        self.jemb = nn.Embedding(jv, k)
        self.eemb = nn.Embedding(ev, k)
        self.lstm1 = nn.LSTM(k, k, num_layers=2,
                             batch_first=True)
        self.lstm2 = nn.LSTM(k, k, num_layers=2,
                             batch_first=True)
        self.Wc = nn.Linear(2*k, k)
        self.W = nn.Linear(k, ev)
        
    def forward(self, jline, eline):

        # encoder
        x = self.jemb(jline)
        ox, (hnx, cnx) = self.lstm1(x)

        # decoder
        y = self.eemb(eline)
        oy, (hny, cny) = self.lstm2(y,(hnx, cnx))

        # attention
        ox1 = ox.permute(0,2,1) # 内積のためにT, Vを転置
        sim = torch.bmm(oy,ox1) # 内積 -> a前のスコア
        bs, yws, xws = sim.shape
        sim2 = sim.reshape(bs*yws,xws)
        alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws) # １つ上のコード、このコードのreshapeをなくして、F.softmax(sim2, dim=2)でよくない？？
        ct = torch.bmm(alpha,ox) # attentionのcontext vec
        oy1 = torch.cat([ct,oy],dim=2)
        oy2 = self.Wc(oy1)
        return self.W(oy2)

In [113]:
demb = 200
net = MyAttNMT(jv, ev, demb).to(device)
optimizer = optim.SGD(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()

# Learn

for epoch in range(20):
    loss1K = 0.0
    for i in tqdm(range(len(jdata))):
        jinput = torch.LongTensor([jdata[i][1:]]).to(device)
        einput = torch.LongTensor([edata[i][:-1]]).to(device)
        out = net(jinput, einput)
        gans = torch.LongTensor([edata[i][1:]]).to(device)
        loss = criterion(out[0],gans[0])
        loss1K += loss.item()
        if (i % 100 == 0):
            # print(epoch, i, loss1K)
            loss1K = 0.0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    outfile = "attnmt-" + str(epoch) + ".model"
    torch.save(net.state_dict(),outfile)


 42%|████▏     | 21221/50000 [01:37<02:11, 218.15it/s]


KeyboardInterrupt: 

In [114]:
# Data setting

class MyDataset(Dataset):
    def __init__(self, xdata, ydata):
        self.data = xdata
        self.label = ydata
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.label[idx]
        return (x,y)

def my_collate_fn(batch):
    images, targets= list(zip(*batch))
    xs = list(images)
    ys = list(targets)
    return xs, ys

###########################

id, eid2w, ew2id = 1, {}, {}
with open('train.en.vocab.4k','r',encoding='utf-8') as f:
    for w in f:
        w = w.strip()
        eid2w[id] = w
        ew2id[w] = id
        id += 1
ev = id

edata = []
with open('train.en','r',encoding='utf-8') as f:
    for sen in f:
        wl = [ew2id['<s>']]
        for w in sen.strip().split():
            if w in ew2id:
                wl.append(ew2id[w])
            else:
                wl.append(ew2id['<unk>'])
        wl.append(ew2id['</s>'])
        edata.append(wl)

id, jid2w, jw2id = 1, {}, {}
with open('train.ja.vocab.4k','r',encoding='utf-8') as f:
    id = 1
    for w in f:
        w = w.strip()
        jid2w[id] = w
        jw2id[w] = id
        id += 1
jv = id

jdata = []
with open('train.ja','r',encoding='utf-8') as f:
    for sen in f:
        wl = [jw2id['<s>']]
        for w in sen.strip().split():
            if w in jw2id:
                wl.append(jw2id[w])
            else:
                wl.append(jw2id['<unk>'])
        wl.append(jw2id['</s>'])
        jdata.append(wl)

batch_size = 100
dataset = MyDataset(jdata,edata)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=my_collate_fn)


# Define model

class MyAttNMT(nn.Module):
    def __init__(self, jv, ev, k):
        super(MyAttNMT, self).__init__()
        self.jemb = nn.Embedding(jv, k, padding_idx=0)
        self.eemb = nn.Embedding(ev, k, padding_idx=0)
        self.lstm1 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.lstm2 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.Wc = nn.Linear(2*k, k)
        self.W = nn.Linear(k, ev)
    def forward(self, jline, eline):
        x = self.jemb(jline)
        ox, (hnx, cnx) = self.lstm1(x)
        y = self.eemb(eline)
        oy, (hny, cny) = self.lstm2(y,(hnx, cnx))
        ox1 = ox.permute(0,2,1)
        sim = torch.bmm(oy,ox1)
        bs, yws, xws = sim.shape
        sim2 = sim.reshape(bs*yws,xws)
        alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws)
        ct = torch.bmm(alpha,ox)
        oy1 = torch.cat([ct,oy],dim=2)
        oy2 = self.Wc(oy1)
        return self.W(oy2)

# model generate, optimizer and criterion setting

demb = 200
net = MyAttNMT(jv, ev, demb).to(device)
optimizer = optim.SGD(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

# Learn

In [122]:
net.train()
for ep in range(20):
    i = 0
    for xs, ys in tqdm(dataloader):
        xs1, ys1, ys2 = [], [], []
        for k in range(len(xs)): # 100バッチ分の繰り返し
            tid = xs[k]
            xs1.append(torch.LongTensor(tid[1:]))
            tid = ys[k]
            ys1.append(torch.LongTensor(tid[:-1]))
            ys2.append(torch.LongTensor(tid[1:]))
        jinput = pad_sequence(xs1, batch_first=True).to(device)
        einput = pad_sequence(ys1, batch_first=True).to(device)
        gans = pad_sequence(ys2, batch_first=True, padding_value=-1.0).to(device)
        out = net(jinput, einput)
        loss = criterion(out[0],gans[0])
        for h in range(1,len(gans)):
            loss += criterion(out[h],gans[h])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1

    print(ep, i, loss.item())
    outfile = "attnmt2-" + str(ep) + ".model"
    torch.save(net.state_dict(),outfile)

100%|██████████| 500/500 [00:15<00:00, 32.38it/s]


0 500 288.3655090332031


100%|██████████| 500/500 [00:15<00:00, 33.19it/s]


1 500 279.2523498535156


100%|██████████| 500/500 [00:15<00:00, 33.11it/s]


...

17 500 87.37112426757812


100%|██████████| 500/500 [00:15<00:00, 32.95it/s]


18 500 85.02765655517578


100%|██████████| 500/500 [00:14<00:00, 33.84it/s]


19 500 63.71935272216797


In [126]:
!python3 './hidden_files/pytorch-nlp01r1/Chapter4/NMT/att-nmt2-test.py' attnmt2-9.model

In [127]:
# blueを使用した評価

# 正解ラベル
labels = []
with open('./test.en', 'r', encoding='utf-8') as f:
    for sen in f:
        w = sen.strip().split()
        labels.append([w])

# モデルが予想した文字列
myans = []
with open('./att-nmt2-output', 'r', encoding='utf-8') as f:
    for sen in f:
        w = sen.strip().split()
        myans.append(w)

score = corpus_bleu(labels, myans)

print(score*100)


16.712199002846656
